In [1]:
import pandas as pd
import jieba
import jieba.analyse
import csv
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pyLDAvis 
import pyLDAvis.sklearn
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import corpora, models, similarities

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\botocore\vendored\requests\packages\urllib3\_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, MutableMapping
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\botocore\vendored\requests\packages\urllib3\_collections.py:1: DeprecationWarning: Using or importing the ABCs fr

In [5]:
df = pd.read_excel("./US_r.xlsx",usecols=[2,2])  
df = df.dropna()
df.to_csv('US_r.txt',sep='\t', index=False)  #轉存txt

In [6]:
jieba.set_dictionary('dict.txt.big')  #插入jieba字典
jieba.load_userdict('userdict.txt')   #插入個人化字典

Building prefix dict from C:\Users\１２３\Desktop\lda2vec\jieba\dict.txt.big ...
Loading model from cache C:\Users\１２３\AppData\Local\Temp\jieba.u6995e702a66797706059f21490f2a8f5.cache
Loading model cost 0.621 seconds.
Prefix dict has been built successfully.


In [7]:
#寫入檔案
input_path='./US_r.txt'
output_path='./US_r_pre.txt'
stopwords_path='./stopword.txt'


# 設定停用詞
stopwords = []
with open(stopwords_path, 'r', encoding='utf-8') as f:
    for line in f:
        if len(line)>0:
            stopwords.append(line.strip())

def tokenizer(s):
    words = []
    cut = jieba.cut(s)
    for word in cut:
        if word not in stopwords:
            words.append(word)
    return words

#讀取檔案資料，分詞，並輸出到檔案
with open(output_path,'w',encoding='utf-8') as o:
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            s=tokenizer(line.strip())
            o.write(" ".join(s)+"\n")

In [2]:
import numpy as np
from gensim import corpora, models


if __name__ == '__main__':
    # 读入文本数据
    f = open('US_r_pre.txt', encoding='utf-8')  # 输入已经预处理后的文本
    texts = [[word for word in line.split()] for line in f]
    f.close()
    M = len(texts)
    print('文本数目：%d 个' % M)

    # 建立词典
    dictionary = corpora.Dictionary(texts)
    V = len(dictionary)
    print('词的个数：%d 个' % V)

    # 计算文本向量
    corpus = [dictionary.doc2bow(text) for text in texts]  # 每个text对应的稀疏向量

    # 计算文档TF-IDF
    corpus_tfidf = models.TfidfModel(corpus)[corpus]

    # LDA模型拟合
    num_topics = 5  # 定义主题数
    lda = models.LdaModel(corpus_tfidf, num_topics=num_topics, id2word=dictionary,
                          alpha=0.01, eta=0.01, minimum_probability=0.001,
                          update_every=1, chunksize=100, passes=1)

    # 所有文档的主题
    doc_topic = [a for a in lda[corpus_tfidf]]
    print('Document-Topic:')
    print(doc_topic)

    # 打印文档的主题分布
    num_show_topic = 15  # 每个文档显示前几个主题
    print('文档的主题分布：')
    doc_topics = lda.get_document_topics(corpus_tfidf)  # 所有文档的主题分布
    idx = np.arange(M)  # M为文本个数，生成从0开始到M-1的文本数组
    for i in idx:
        topic = np.array(doc_topics[i])
        topic_distribute = np.array(topic[:, 1])
        topic_idx = topic_distribute.argsort()[:-num_show_topic - 1:-1]  # 按照概率大小进行降序排列
        print('第%d个文档的前%d个主题：' % (i, num_show_topic))
        print(topic_idx)
        print(topic_distribute[topic_idx])

    # 每个主题的词分布
    num_show_term = 30  # 每个主题显示几个词
    for topic_id in range(num_topics):
        print('主题#%d：\t' % topic_id)
        term_distribute_all = lda.get_topic_terms(topicid=topic_id)  # 所有词的词分布
        term_distribute = term_distribute_all[:num_show_term]  # 只显示前几个词
        term_distribute = np.array(term_distribute)
        term_id = term_distribute[:, 0].astype(np.int)
        print('词：', end="")
        for t in term_id:
            print(dictionary.id2token[t], end=' ')
        print('概率：', end="")
        print(term_distribute[:, 1])

    with open('topicword.txt', 'w', encoding='utf-8') as tw:
        for topic_id in range(num_topics):
            term_distribute_all = lda.get_topic_terms(topicid=topic_id, topn=70)
            term_distribute = np.array(term_distribute_all)
            term_id = term_distribute[:, 0].astype(np.int)
            for t in term_id:
                tw.write(dictionary.id2token[t] + " ")
            tw.write("\n")

文本数目：12332 个
词的个数：35118 个
Document-Topic:
[[(0, 0.2), (1, 0.2), (2, 0.2), (3, 0.2), (4, 0.2)], [(0, 0.014209661), (1, 0.9431613), (2, 0.014209661), (3, 0.014209661), (4, 0.014209661)], [(0, 0.50340265), (1, 0.4773256), (2, 0.0064239325), (3, 0.0064239325), (4, 0.0064239325)], [(0, 0.010763026), (1, 0.95694786), (2, 0.010763026), (3, 0.010763026), (4, 0.010763026)], [(0, 0.13707288), (1, 0.8378816), (2, 0.008348524), (3, 0.008348524), (4, 0.008348524)], [(0, 0.22270352), (1, 0.52592015), (2, 0.2385371), (3, 0.0064196256), (4, 0.0064196256)], [(0, 0.46278387), (1, 0.52526367), (2, 0.0039841593), (3, 0.0039841593), (4, 0.0039841593)], [(0, 0.0036262826), (1, 0.9854949), (2, 0.0036262826), (3, 0.0036262826), (4, 0.0036262826)], [(0, 0.0026315313), (1, 0.9894739), (2, 0.0026315313), (3, 0.0026315313), (4, 0.0026315313)], [(0, 0.012704998), (1, 0.4091627), (2, 0.012704998), (3, 0.5527223), (4, 0.012704998)], [(0, 0.14708026), (1, 0.6903173), (2, 0.14908609), (3, 0.006758181), (4, 0.006758181

[1 4 3 2 0]
[0.96654624 0.00836345 0.00836345 0.00836345 0.00836345]
第32个文档的前15个主题：
[0 1 4 3 2]
[0.83403379 0.15756953 0.00279888 0.00279888 0.00279888]
第33个文档的前15个主题：
[0 1 4 3 2]
[0.51254624 0.4764556  0.00366604 0.00366604 0.00366604]
第34个文档的前15个主题：
[1 0 4 3 2]
[0.50161135 0.34176525 0.14348778 0.00656781 0.00656781]
第35个文档的前15个主题：
[1 3 4 2 0]
[0.75801522 0.21159773 0.01012902 0.01012902 0.01012902]
第36个文档的前15个主题：
[1 0 2 4 3]
[0.50270319 0.26555526 0.22290193 0.00441979 0.00441979]
第37个文档的前15个主题：
[1 3 4 2 0]
[0.84380823 0.14054002 0.00521725 0.00521725 0.00521725]
第38个文档的前15个主题：
[0 4 3 2 1]
[0.97985685 0.00503579 0.00503579 0.00503579 0.00503579]
第39个文档的前15个主题：
[1 4 3 2 0]
[0.97575688 0.00606079 0.00606079 0.00606079 0.00606079]
第40个文档的前15个主题：
[4 1 0 3 2]
[0.46569514 0.39291111 0.13150363 0.00494508 0.00494508]
第41个文档的前15个主题：
[1 4 3 2 0]
[0.94117796 0.01470551 0.01470551 0.01470551 0.01470551]
第42个文档的前15个主题：
[1 4 3 2 0]
[0.95431811 0.01142047 0.01142047 0.01142047 0.01142047]
第43个文档的

第158个文档的前15个主题：
[1 4 0 3 2]
[0.73765165 0.20111701 0.05372385 0.00375376 0.00375376]
第159个文档的前15个主题：
[4 1 0 3 2]
[0.48127079 0.39447126 0.10194337 0.01115729 0.01115729]
第160个文档的前15个主题：
[1 0 4 3 2]
[0.82498789 0.16553505 0.003159   0.003159   0.003159  ]
第161个文档的前15个主题：
[1 0 4 3 2]
[0.79759681 0.18978903 0.00420472 0.00420472 0.00420472]
第162个文档的前15个主题：
[1 0 4 3 2]
[0.90999353 0.07309514 0.00563712 0.00563712 0.00563712]
第163个文档的前15个主题：
[1 0 4 3 2]
[0.69179654 0.29136598 0.0056125  0.0056125  0.0056125 ]
第164个文档的前15个主题：
[1 0 4 3 2]
[0.72485107 0.26030245 0.00494884 0.00494884 0.00494884]
第165个文档的前15个主题：
[1 4 3 2 0]
[0.96860582 0.00784855 0.00784855 0.00784855 0.00784855]
第166个文档的前15个主题：
[1 0 4 3 2]
[0.85594559 0.13235067 0.00390125 0.00390125 0.00390125]
第167个文档的前15个主题：
[1 0 4 3 2]
[0.87157416 0.11848878 0.00331236 0.00331236 0.00331236]
第168个文档的前15个主题：
[1 4 3 2 0]
[0.91997486 0.02000629 0.02000629 0.02000629 0.02000629]
第169个文档的前15个主题：
[1 0 4 3 2]
[0.68014842 0.17302488 0.13987146 0.0

第282个文档的前15个主题：
[1 0 2 4 3]
[0.73120433 0.19500987 0.06582607 0.00397986 0.00397986]
第283个文档的前15个主题：
[1 0 4 3 2]
[0.77983737 0.19387239 0.00876342 0.00876342 0.00876342]
第284个文档的前15个主题：
[1 0 4 3 2]
[0.78369659 0.20546407 0.00361311 0.00361311 0.00361311]
第285个文档的前15个主题：
[1 0 4 3 2]
[0.78369641 0.20546426 0.00361311 0.00361311 0.00361311]
第286个文档的前15个主题：
[1 4 0 3 2]
[0.6217792  0.20365913 0.16215795 0.00620186 0.00620186]
第287个文档的前15个主题：
[4 1 3 2 0]
[0.51115853 0.46632117 0.00750676 0.00750676 0.00750676]
第288个文档的前15个主题：
[1 0 4 3 2]
[0.80452746 0.1741583  0.00710474 0.00710474 0.00710474]
第289个文档的前15个主题：
[1 0 4 3 2]
[0.57419538 0.23247679 0.18372577 0.00480103 0.00480103]
第290个文档的前15个主题：
[1 0 4 3 2]
[0.56882018 0.22907782 0.19233887 0.00488156 0.00488156]
第291个文档的前15个主题：
[1 0 4 3 2]
[0.6514284  0.33104342 0.00584272 0.00584272 0.00584272]
第292个文档的前15个主题：
[1 4 0 3 2]
[0.72348028 0.15087552 0.11799218 0.00382599 0.00382599]
第293个文档的前15个主题：
[1 4 0 3 2]
[0.72348034 0.15087554 0.11799215 0.0

第389个文档的前15个主题：
[0 4 3 2 1]
[0.89198995 0.02700251 0.02700251 0.02700251 0.02700251]
第390个文档的前15个主题：
[0 1 4 3 2]
[0.53532726 0.43456888 0.01003464 0.01003464 0.01003464]
第391个文档的前15个主题：
[1 0 4 3 2]
[0.64292657 0.2111267  0.13427873 0.005834   0.005834  ]
第392个文档的前15个主题：
[1 0 4 3 2]
[0.68375474 0.29882175 0.00580783 0.00580783 0.00580783]
第393个文档的前15个主题：
[1 0 4 3 2]
[0.82082999 0.16197816 0.00573061 0.00573061 0.00573061]
第394个文档的前15个主题：
[0 1 4 3 2]
[0.53566378 0.4334074  0.01030961 0.01030961 0.01030961]
第395个文档的前15个主题：
[1 4 0 3 2]
[0.75337523 0.12569568 0.1079411  0.00649398 0.00649398]
第396个文档的前15个主题：
[1 4 3 2 0]
[0.9750827  0.00622933 0.00622933 0.00622933 0.00622933]
第397个文档的前15个主题：
[1 0 4 3 2]
[0.54264176 0.2683309  0.16595942 0.01153395 0.01153395]
第398个文档的前15个主题：
[1 0 4 3 2]
[0.54264301 0.2683304  0.16595872 0.01153394 0.01153394]
第399个文档的前15个主题：
[1 4 3 2 0]
[0.95122272 0.01219431 0.01219431 0.01219431 0.01219431]
第400个文档的前15个主题：
[1 0 3 4 2]
[0.72993487 0.16020545 0.10002897 0.0

...

第11758个文档的前15个主题：
[0 4 3 2 1]
[0.91633719 0.02091571 0.02091571 0.02091571 0.02091571]
第11759个文档的前15个主题：
[1 0 4 3 2]
[0.53437257 0.42313117 0.01416543 0.01416543 0.01416543]
第11760个文档的前15个主题：
[1 4 3 2 0]
[0.92498916 0.0187527  0.0187527  0.0187527  0.0187527 ]
第11761个文档的前15个主题：
[4 3 2 1 0]
[0.2 0.2 0.2 0.2 0.2]
第11762个文档的前15个主题：
[1 4 3 2 0]
[0.90236437 0.02440892 0.02440892 0.02440892 0.02440892]
第11763个文档的前15个主题：
[4 3 2 1 0]
[0.2 0.2 0.2 0.2 0.2]
第11764个文档的前15个主题：
[1 4 3 2 0]
[0.92113203 0.01971698 0.01971698 0.01971698 0.01971698]
第11765个文档的前15个主题：
[1 4 3 2 0]
[0.91646922 0.0208827  0.0208827  0.0208827  0.0208827 ]
第11766个文档的前15个主题：
[1 4 3 2 0]
[0.65380305 0.3186858  0.00917038 0.00917038 0.00917038]
第11767个文档的前15个主题：
[4 3 2 1 0]
[0.2 0.2 0.2 0.2 0.2]
第11768个文档的前15个主题：
[1 4 3 2 0]
[0.92230099 0.01942473 0.01942473 0.01942473 0.01942473]
第11769个文档的前15个主题：
[1 4 3 2 0]
[0.92280346 0.01929914 0.01929914 0.01929914 0.01929914]
第11770个文档的前15个主题：
[1 0 4 3 2]
[0.81415218 0.16813898 0.005902

第11866个文档的前15个主题：
[1 4 3 2 0]
[0.92506945 0.01873264 0.01873264 0.01873264 0.01873264]
第11867个文档的前15个主题：
[1 4 3 2 0]
[0.96339506 0.00915125 0.00915125 0.00915125 0.00915125]
第11868个文档的前15个主题：
[4 3 2 1 0]
[0.2 0.2 0.2 0.2 0.2]
第11869个文档的前15个主题：
[1 4 3 2 0]
[0.90818167 0.02295459 0.02295459 0.02295459 0.02295459]
第11870个文档的前15个主题：
[1 4 3 2 0]
[0.90103984 0.02474004 0.02474004 0.02474004 0.02474004]
第11871个文档的前15个主题：
[4 3 2 1 0]
[0.2 0.2 0.2 0.2 0.2]
第11872个文档的前15个主题：
[4 3 2 1 0]
[0.2 0.2 0.2 0.2 0.2]
第11873个文档的前15个主题：
[4 3 2 1 0]
[0.93312711 0.01671821 0.01671821 0.01671821 0.01671821]
第11874个文档的前15个主题：
[4 3 2 1 0]
[0.2 0.2 0.2 0.2 0.2]
第11875个文档的前15个主题：
[0 4 3 2 1]
[0.8561464  0.03596339 0.03596339 0.03596339 0.03596339]
第11876个文档的前15个主题：
[1 4 3 2 0]
[0.94449657 0.01387585 0.01387585 0.01387585 0.01387585]
第11877个文档的前15个主题：
[1 4 3 2 0]
[0.89144796 0.02713802 0.02713802 0.02713802 0.02713802]
第11878个文档的前15个主题：
[0 4 3 2 1]
[0.96190464 0.00952385 0.00952385 0.00952385 0.00952385]
第11879个文档

[0.92391968 0.01902009 0.01902009 0.01902009 0.01902009]
第12053个文档的前15个主题：
[1 4 3 2 0]
[0.81717277 0.0457068  0.0457068  0.0457068  0.0457068 ]
第12054个文档的前15个主题：
[4 3 2 1 0]
[0.2 0.2 0.2 0.2 0.2]
第12055个文档的前15个主题：
[0 4 3 2 1]
[0.94991946 0.01252012 0.01252012 0.01252012 0.01252012]
第12056个文档的前15个主题：
[0 4 3 2 1]
[0.84432834 0.03891791 0.03891791 0.03891791 0.03891791]
第12057个文档的前15个主题：
[1 4 3 2 0]
[0.78348333 0.19700356 0.00650436 0.00650436 0.00650436]
第12058个文档的前15个主题：
[0 4 3 2 1]
[0.96780694 0.00804828 0.00804828 0.00804828 0.00804828]
第12059个文档的前15个主题：
[2 1 4 3 0]
[0.49720681 0.46684828 0.01198163 0.01198163 0.01198163]
第12060个文档的前15个主题：
[1 4 3 2 0]
[0.90904474 0.02273882 0.02273882 0.02273882 0.02273882]
第12061个文档的前15个主题：
[0 4 3 2 1]
[0.93264604 0.01683848 0.01683848 0.01683848 0.01683848]
第12062个文档的前15个主题：
[4 3 2 1 0]
[0.2 0.2 0.2 0.2 0.2]
第12063个文档的前15个主题：
[2 1 4 3 0]
[0.67223561 0.26444849 0.02110529 0.02110529 0.02110529]
第12064个文档的前15个主题：
[1 4 3 2 0]
[0.63132662 0.34093875 0.0

[0 4 3 2 1]
[0.93830168 0.01542458 0.01542458 0.01542458 0.01542458]
第12184个文档的前15个主题：
[4 3 2 1 0]
[0.2 0.2 0.2 0.2 0.2]
第12185个文档的前15个主题：
[0 4 3 2 1]
[0.89415884 0.02646029 0.02646029 0.02646029 0.02646029]
第12186个文档的前15个主题：
[1 0 4 3 2]
[0.72299135 0.2610057  0.00533432 0.00533432 0.00533432]
第12187个文档的前15个主题：
[0 4 3 2 1]
[0.93887568 0.01528107 0.01528107 0.01528107 0.01528107]
第12188个文档的前15个主题：
[1 0 3 4 2]
[0.59035337 0.2872439  0.11180037 0.00530116 0.00530116]
第12189个文档的前15个主题：
[4 1 3 2 0]
[0.59854066 0.37375453 0.00923494 0.00923494 0.00923494]
第12190个文档的前15个主题：
[4 3 2 1 0]
[0.2 0.2 0.2 0.2 0.2]
第12191个文档的前15个主题：
[1 4 3 2 0]
[0.93245816 0.01688546 0.01688546 0.01688546 0.01688546]
第12192个文档的前15个主题：
[1 4 3 2 0]
[0.94274306 0.01431422 0.01431422 0.01431422 0.01431422]
第12193个文档的前15个主题：
[1 4 3 2 0]
[0.91614336 0.02096416 0.02096416 0.02096416 0.02096416]
第12194个文档的前15个主题：
[0 4 3 2 1]
[0.92552161 0.01861961 0.01861961 0.01861961 0.01861961]
第12195个文档的前15个主题：
[0 4 3 2 1]
[0.89764959 0.

[1 4 3 2 0]
[0.8395403  0.04011492 0.04011492 0.04011492 0.04011492]
第12313个文档的前15个主题：
[1 4 3 2 0]
[0.8398326  0.04004184 0.04004184 0.04004184 0.04004184]
第12314个文档的前15个主题：
[4 3 2 1 0]
[0.2 0.2 0.2 0.2 0.2]
第12315个文档的前15个主题：
[1 4 3 2 0]
[0.77645296 0.05588676 0.05588676 0.05588676 0.05588676]
第12316个文档的前15个主题：
[0 4 3 2 1]
[0.9036147  0.02409633 0.02409633 0.02409633 0.02409633]
第12317个文档的前15个主题：
[4 3 2 1 0]
[0.2 0.2 0.2 0.2 0.2]
第12318个文档的前15个主题：
[4 3 2 1 0]
[0.2 0.2 0.2 0.2 0.2]
第12319个文档的前15个主题：
[1 0 4 3 2]
[0.53864551 0.42591193 0.01181419 0.01181419 0.01181419]
第12320个文档的前15个主题：
[1 4 3 2 0]
[0.86188883 0.03452779 0.03452779 0.03452779 0.03452779]
第12321个文档的前15个主题：
[1 4 3 2 0]
[0.91681463 0.02079634 0.02079634 0.02079634 0.02079634]
第12322个文档的前15个主题：
[1 4 3 2 0]
[0.95985216 0.01003696 0.01003696 0.01003696 0.01003696]
第12323个文档的前15个主题：
[0 1 4 3 2]
[0.51240331 0.46994573 0.00588365 0.00588365 0.00588365]
第12324个文档的前15个主题：
[1 4 0 3 2]
[0.81627005 0.09044546 0.08736905 0.00295772 0.00

In [3]:
from gensim.models import CoherenceModel


print('Perplexity: ', lda.log_perplexity(corpus))


coherence_model_lda = CoherenceModel(model=lda, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda) # 主题一致性得分

Perplexity:  -70.17224203456637
Coherence Score:  0.42742158935062413


In [4]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
LDAvis_prepared


c:\users\user\appdata\local\programs\python\python37\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.354141  0.187116       1        1  72.277679
0     -0.153428 -0.303525       2        1  17.527925
4      0.167483  0.037620       3        1   6.205907
2      0.165486  0.039651       4        1   2.520943
3      0.174600  0.039139       5        1   1.467554, topic_info=     Category         Freq Term        Total  loglift  logprob
697   Default  1956.000000   支那  1956.000000  30.0000  30.0000
572   Default   649.000000    蛆   649.000000  29.0000  29.0000
771   Default  1005.000000   航母  1005.000000  28.0000  28.0000
678   Default   843.000000  全世界   843.000000  27.0000  27.0000
774   Default  1306.000000   高潮  1306.000000  26.0000  26.0000
...       ...          ...  ...          ...      ...      ...
731    Topic5     0.064163   繞著     0.870544   1.6139 -11.0415
985    Topic5     0.064159   十年     0.870539   1.6138 -11.0415
1996   Topic5     0.064156  老百姓     0.870537   1.6138 -11.0416
4      Topic5     0.064172   不用   502.195709  -4.7436 -11.0413
209    Topic5     0.064165   領空  1690.846802  -5.9577 -11.0414

[213 rows x 6 columns], token_table=      Topic      Freq Term
term                      
2371      3  0.994627   下水
81        1  0.918750    不
81        2  0.081186    不
632       3  0.999105   不想
856       1  0.977042   不會
...     ...       ...  ...
881       2  0.000489   飛彈
18        1  0.999598   飛機
522       5  0.991684   飛過
2696      3  0.998719  驅逐艦
774       2  0.999269   高潮

[191 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 5, 3, 4])

In [85]:
import gensim.models as word2vec
import numpy as np
from gensim.models.word2vec import LineSentence


class TextRank(object):

    def __init__(self, simvalue, alpha, iter_num):
        self.word_list = []  # 记录主题-词模型
        self.edge_dict = {}  # 记录节点的边连接字典
        self.simvalue = simvalue  # 满足该最小相似性值，词与词之间可构成边
        self.alpha = alpha
        self.iter_num = iter_num  # 迭代次数

    # 读取文档，返回一个总的主题-词列表
    def readFile(self):
        with open("topicword.txt", "r", encoding='utf-8') as tw:
            for line in tw:
                self.word_list.append(line.strip().split(" "))
        # print(self.word_list)
        return self.word_list

    # 加载Word2Vec模型
    def loadModel(self):
        self.path = "word2vec.model"  # Word2Vec模型路径
        self.model = word2vec.Word2Vec.load(self.path)
        print("模型加载完成")
        return self.model

    # 计算TR值
    def calTR(self):

        # 首先根据词语之间相似性，构建每个词的相邻词,过滤掉相似性较小的词，返回边的集合
        word_list_len = len(self.word_list)  # 主题个数
        term_num = 30  # 每个主题下词的个数
        names = globals()
        for list, t in zip(self.word_list, range(word_list_len)):  # list表示每一个主题-词，t为主题序数，即第几个主题
            names['self.edge_dict_' + str(t)] = {}  # 为每一个主题建立一个词典，如第一个主题的词典名称为 self.edge_dict_0
            for index, word in enumerate(list):  # 枚举可以同时遍历索引和值
                if word not in names.get('self.edge_dict_' + str(t)).keys():  # 表明该词还未进行边的连接
                    tmp_set = set()  # set()函数创建一个无序不重复元素集
                    for i in range(term_num):
                        if i == index:
                            continue  # 若为该单词，则跳出本次循环
                        word0 = list[i]
                        similarity = self.model.wv.similarity(word, word0)  # 计算当前词与剩余词语的相似性
                        if similarity > self.simvalue:
                            tmp_set.add(word0)
                    #  返回一个词的相邻节点
                    names.get('self.edge_dict_' + str(t))[word] = tmp_set
            # print(names.get('self.edge_dict_' + str(t)))

        # 根据边的相连关系，构建矩阵
        # for list, t in zip(self.word_list, range(word_list_len)):
            # 用0来初始化矩阵
            names['self.matrix_' + str(t)] = np.zeros([len(set(list)), len(set(list))])
            self.word_index = {}  # 记录词的index
            self.index_dict = {}  # 记录节点index对应的词
            for i, v in enumerate(set(list)):
                self.word_index[v] = i  # 用词来找到它的索引
                self.index_dict[i] = v  # 用索引来找到它的词
            for key in names.get('self.edge_dict_' + str(t)).keys():  # 对于字典里的每一个节点
                for w in names.get('self.edge_dict_' + str(t))[key]:  # 对于每一个节点的相邻点
                    # 相邻节点的边权重变为向量相似度
                    names.get('self.matrix_' + str(t))[self.word_index[key]][self.word_index[w]] = self.model.wv.similarity(key, w)
                    names.get('self.matrix_' + str(t))[self.word_index[w]][self.word_index[key]] = self.model.wv.similarity(w, key)
            # print(names.get('self.matrix_' + str(t)))

        # 归一化
            for j in range(names.get('self.matrix_' + str(t)).shape[1]):  # 对第j列元素
                sum = 0
                for i in range(names.get('self.matrix_' + str(t)).shape[0]):  # 对第i行元素
                    sum += names.get('self.matrix_' + str(t))[i][j]
                for i in range(names.get('self.matrix_' + str(t)).shape[0]):
                    names.get('self.matrix_' + str(t))[i][j] /= sum


        # 根据textrank公式计算权重
        # for list, t in zip(self.word_list, range(word_list_len)):
            self.TR = np.ones([len(set(list)), 1])
            for i in range(self.iter_num):
                self.TR = (1 - self.alpha) + self.alpha * np.dot(names.get('self.matrix_' + str(t)), self.TR)
            print("主题#%d:" % t)
            #print(self.TR)

            # 输出词和相应的权重
            word_pr = {}
            for i in range(len(self.TR)):
                word_pr[self.index_dict[i]] = self.TR[i][0]
            # sorted() 函数对所有可迭代的对象进行排序操作，返回的是一个新的 list，key指用来进行比较的元素，reverse为排序规则  ，True降序 ，False升序（默认）
            res = sorted(word_pr.items(), key=lambda x: x[1], reverse=True)
            # print(res)
            list = []
            for n in range(len(res)):
                list.append(res[n][0])
            print(list)


if __name__ == '__main__':
    tr = TextRank(0.60, 0.90, 1000)
    tr.readFile()
    tr.loadModel()
    tr.calTR()


c:\users\１２３\appdata\local\programs\python\python37\lib\site-packages\smart_open\smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


模型加载完成
主题#0:
['世界', '共軍', '南海', '航行', '飛機', '警察', '國家', '公海', '中國', '軍機', '穿越', '美國軍艦', '不會', '共產黨', '自由', '民主', '領空', '大陸', '核彈', '中華民國', '監控', '有種', '隨便', '美軍', '支那', '飛彈', '巡弋', '戰爭', '我國', '如果', '領海', '日本', '海域', '北韓', '主權', '美國', '需要', '軍隊', '只會', '邪惡', '全世界', '軍艦', '中共']
主题#1:
['一點', '常識', '抗議', '打下來', '洗腦', '主子', '軍火', '嘴砲', '加油', '感覺', '中國', '戰斧', '賣', '拜託', '笨蛋', '心態', '靠近', '地方', '保衛', '每次', '敘利亞', '高潮', '還在', '伊拉克', '國人', '開放', '不知', '內部', '只要', '共機', '囂張', '國土', '參加', '先進', '外海', '發射', '樣子', '美國', '好處', '多久', '巡弋飛彈', '離開']
主题#2:
['航行', '祖國', '軍費', '唯獨', '島礁', '中國', '不想', '繞行', '幫', '飛過', '自由', '錯誤', '屁事', '大陸', '只要', '中國解放軍', '不行', '海面', '挑釁', '美軍', '花錢', '只敢', '例行', '不用', '沒用', '保護', '海域', '犧牲', '主權', '降落', '美國', '飛越', '只會', '反正', '夏威夷', '不見', '軍艦']


c:\users\１２３\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:75: RuntimeWarning: invalid value encountered in double_scalars


主题#3:
['南海', '執行', '民進黨', '機會', '轟炸機', '進出', '可憐', '穿越', '動武', '霸凌', '空域', '政客', '肯定', '只要', '這是', '出動', '共機', '太平洋', '國民黨', '這種', '挑釁', '美軍', '消息', '爸爸', '執政', '事情', '開打', '東海', '留言', '亞洲', '軍艦', '統一', '美帝', '區域', '演習', '報導', '主動', '獨立', '停止']
主题#4:
['國防', '航母', '民進黨', '飛機', '國家', '開戰', '支持', '早就', '中國', '好像', '可能', '利益', '艦隊', '不會', '大陸', '擁有', '中美', '錢', '武器', '美軍', '飛彈', '買', '國防部', '其實', '五毛', '保護', '如果', '全程', '垃圾', '國內', '國軍', '中華', '美國', '習近平', '新聞', '川普', '保護費', '臺灣', '只會', '獨立', '低能', '幹', '中共']


NameError: name 'lda_model' is not defined

In [30]:
#tf-idf
#with open(output_path, 'r', encoding = 'utf-8') as file:
#    text = file.readlines()
#   keywords = jieba.analyse.extract_tags(str(text), topK = 1000, withWeight=True, allowPOS=())
#   print(keywords)

In [10]:
#主題模型圖表
def print_top_words(model, feature_names, n_top_words):
    for topic_idx,topic in enumerate(model.components_):
        print("Topic #%d:"%topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]))
        print()
#資料視覺化
data = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
pyLDAvis.show(data)

UnicodeEncodeError: 'ascii' codec can't encode characters in position 18-20: ordinal not in range(128)

In [ ]:
#Word2Vec
#主題模型建構
#載入套件
n_features = 5000
n_top_words = 200
n_topics = 10

with open(output_path, 'rb+') as file:
    text = file.readlines()
    tf_vectorizer = CountVectorizer(strip_accents ='unicode',
                                max_features=n_features, stop_words='english', max_df=0.5,min_df=10)
    tf=tf_vectorizer.fit_transform(text)
    lda =  LatentDirichletAllocation(n_components=n_topics, max_iter=50, learning_method='online', learning_offset=50,random_state=0)
    lda.fit(tf)
    print()

In [ ]:
feature_names = tf_vectorizer.get_feature_names()
compoments = lda.components_
tf_feature_names =tf_vectorizer.get_feature_names()
#匯出至excel 
topic_dic = {}
for no in range(n_topics):
    topic = ([feature_names[i] for i in compoments[no].argsort()[:-n_top_words - 1:-1]])
    topic_dic["topic"+str(no+1)] = topic


topic_df = pd.DataFrame(topic_dic)
topic_df.to_excel("result.xlsx",index=False)